In [21]:
from __future__ import print_function

import numpy as np

import scipy.misc
import scipy.io

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.models import load_model, Model, Sequential
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Lambda, Flatten, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Reshape
from keras.layers.merge import concatenate
from keras import optimizers
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image
import random
import os
import h5py

%matplotlib inline

In [22]:
BASE_DIR = "./ships-in-satellite-imagery/shipsnet/"
TRAIN_ROWS = .8
IMAGE_HEIGHT = 80
IMAGE_WIDTH = 80
COLOR_CHANNELS = 3

In [23]:
def readFromImages():
    file_list = os.listdir(BASE_DIR)
    total_images = len(file_list)
    file_list = random.sample(file_list, total_images)

    x_data = np.zeros((total_images, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS))
    y_data = np.zeros((total_images, 1))

    for f in range(total_images):
        filename = file_list[f]
        x_data[f] = plt.imread(BASE_DIR + filename, "r")
        y_data[f] = int(filename.split("_", 1)[0])

    h5File = h5py.File("data.h5", "w")
    h5File.create_dataset("x_data", data=x_data)
    h5File.create_dataset("y_data", data=y_data)
    h5File.close()
    
    return x_data, y_data

In [24]:
def readFromFile():
    h5File = h5py.File("data.h5")
    x_data = h5File["x_data"][:]
    y_data = h5File["y_data"][:]
            
    return x_data, y_data

In [25]:
def add_shortcut(inp, shortcut):
    
    # channels for shortcut
    nb_channels = int(shortcut.get_shape()[-1])
    
    # shortcut revived
    shortcut = Conv2D(nb_channels, (3, 3), padding="same")(shortcut)
    shortcut = BatchNormalization()(shortcut)
    return concatenate([shortcut, inp])

In [26]:
input_image = Input(shape=(80, 80, 3), name="input_1")

x = Conv2D(32, (3, 3), name="conv_1", padding="same")(input_image)
x = BatchNormalization(name='norm_1')(x)
x = LeakyReLU(alpha=0.9, name="leaky_relu_1")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_1")(x)

x = Conv2D(64, (3, 3), name="conv_2", padding="same")(x)
x = BatchNormalization(name='norm_2')(x)
x = LeakyReLU(alpha=0.9, name="leaky_relu_2")(x)

x = Conv2D(64, (3, 3), name="conv_3", padding="same")(x)
x = BatchNormalization(name='norm_3')(x)
x = LeakyReLU(alpha=0.9, name="leaky_relu_3")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_2")(x)

x = Conv2D(128, (3, 3), name="conv_4", padding="same")(x)
x = BatchNormalization(name='norm_4')(x)
x = LeakyReLU(alpha=0.9, name="leaky_relu_4")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_3")(x)

# shortcut
shortcut = x

x = Conv2D(64, (3, 3), name="conv_5", padding="same")(x)
x = BatchNormalization(name='norm_5')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_5")(x)
# x = MaxPooling2D(pool_size=(2, 2), name="max_pool_4")(x)

# add shortcut
x = add_shortcut(x, shortcut)

x = Conv2D(32, (3, 3), name="conv_6", padding="same")(x)
x = BatchNormalization(name='norm_6')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_6")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_5")(x)

x = Conv2D(16, (3, 3), name="conv_7", padding="same")(x)
x = BatchNormalization(name='norm_7')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_7")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_6")(x)

x = Conv2D(8, (3, 3), name="conv_8", padding="same")(x)
x = BatchNormalization(name='norm_8')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_8")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_7")(x)

x = Conv2D(4, (3, 3), name="conv_9", padding="same")(x)
x = BatchNormalization(name='norm_9')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_9")(x)
# x = MaxPooling2D(pool_size=(2, 2), name="max_pool_8")(x)

x = Flatten(name="flatten_1")(x)
x = Dense(1, name="dense_1")(x)

model = Model(inputs=input_image, outputs=x)

# model.summary()

In [27]:
# For a binary classification problem
adam_optimizer = optimizers.Adam(lr=0.002, beta_1=0.905, beta_2=0.995, epsilon=1e-08, decay=0.001)

model.compile(optimizer=adam_optimizer,
              loss='mean_squared_logarithmic_error',
              metrics=['accuracy'])

In [28]:
x_data, y_data = readFromFile()
total_images = x_data.shape[0]

In [30]:
# Train the model, iterating on the data in batches of 32 samples
train_rows = int(total_images*TRAIN_ROWS)

num_of_iters = 4

for i in range(num_of_iters):
    randomized_image_indices = random.sample(range(total_images), total_images)
    randomised_train_rows = randomized_image_indices[:train_rows]
    
    x_train = x_data[randomised_train_rows,:,:,:]
    y_train = y_data[randomised_train_rows,:]

    model.fit(x_train, y_train, epochs=i, batch_size=32)

Epoch 1/1
2240/2240 [==============================] - 194s 86ms/step - loss: 0.0549 - acc: 0.8496
Epoch 1/2
2240/2240 [==============================] - 178s 80ms/step - loss: 0.0209 - acc: 0.9321
Epoch 2/2
2240/2240 [==============================] - 197s 88ms/step - loss: 0.0149 - acc: 0.9585
Epoch 1/3
2240/2240 [==============================] - 218s 97ms/step - loss: 0.0128 - acc: 0.9647
Epoch 2/3
2240/2240 [==============================] - 219s 98ms/step - loss: 0.0125 - acc: 0.9594
Epoch 3/3
2240/2240 [==============================] - 236s 106ms/step - loss: 0.0095 - acc: 0.9732


In [31]:
x_test = x_data[train_rows:,:,:,:]
y_test = y_data[train_rows:,:]

model.evaluate(x_test, y_test)

560/560 [==============================] - 19s 34ms/step


[0.0079736878895866024, 0.97499999999999998]

In [32]:
model.save("shipRecognitionModel.h5")

In [32]:
# test_model = load_model("shipRecognitionModel.h5")

In [33]:
# test_model.evaluate(x_test, y_test)

560/560 [==============================] - 19s 33ms/step


[0.0099144536103787165, 0.97857142857142854]